In [100]:
import pymongo
from pymongo import MongoClient
client=MongoClient("mongodb://localhost:27017/")
db=client.companies

In [55]:
collection = db['companies']
comp = collection.find( {
    '$and' : [
        { '$or' : [ { 'number_of_employees' : {'$gt':1000}}] },
        { '$or' : [ { 'founded_year' : {'$lt':2000} } ] },
        { '$or' : [ { 'acquisition.price_currency_code' : 'USD'}] },
        { '$or' : [ { 'acquisition.price_amount' : {'$gte':10000000} } ] },]
        
},{'name':1, '_id':0, 'offices':1} )


In [56]:
import pandas as pd 
df = pd.DataFrame(comp)
df.head()

,name,offices
0,PayPal,"[{'description': '', 'address1': '2145 E Hamil..."
1,Sun Microsystems,"[{'description': None, 'address1': '4150 Netwo..."
2,Zappos,"[{'description': 'Headquarters', 'address1': '..."
3,Razorfish,"[{'description': 'Chicago', 'address1': '600 W..."
4,Blue Coat Systems,"[{'description': None, 'address1': '420 N. Mar..."


In [119]:
df=df.dropna()
def get_first(data):
    data=data['offices']
    principal=None
    if data[0]['latitude'] and data[0]['longitude']:
        principal={'type':'Point',
                  'coordinates':[data[0]['longitude'],
                                 data[0]['latitude']]}
    return {'totalOffices':len(data),
           'lat':data[0]['latitude'],
           'lng':data[0]['longitude'],
            'oficina_principal':principal}
df

,name,offices
0,PayPal,"[{'description': '', 'address1': '2145 E Hamil..."
1,Sun Microsystems,"[{'description': None, 'address1': '4150 Netwo..."
2,Zappos,"[{'description': 'Headquarters', 'address1': '..."
3,Razorfish,"[{'description': 'Chicago', 'address1': '600 W..."
4,Blue Coat Systems,"[{'description': None, 'address1': '420 N. Mar..."
5,LSI,"[{'description': 'HQ', 'address1': '1621 Barbe..."
6,VMware,"[{'description': None, 'address1': '3401 Hillv..."
7,Xerox,"[{'description': 'HQ', 'address1': '45 Glover ..."
8,Novell,"[{'description': '', 'address1': '404 Wyman St..."
9,Kenexa,"[{'description': 'HQ', 'address1': '650 East S..."


In [58]:
off=df[['offices']].apply(get_first, result_type='expand', axis=1)
off.dropna()

,totalOffices,lat,lng,oficina_principal
0,1.0,37.294465,-121.927696,"{'type': 'Point', 'coordinates': [-121.927696,..."
1,1.0,37.394791,-121.955463,"{'type': 'Point', 'coordinates': [-121.955463,..."
2,2.0,36.032976,-115.064859,"{'type': 'Point', 'coordinates': [-115.064859,..."
3,2.0,41.881928,-87.644547,"{'type': 'Point', 'coordinates': [-87.6445473,..."
4,1.0,37.390753,-122.039865,"{'type': 'Point', 'coordinates': [-122.039865,..."
5,1.0,37.405485,-121.910305,"{'type': 'Point', 'coordinates': [-121.910305,..."
6,1.0,37.403315,-122.145981,"{'type': 'Point', 'coordinates': [-122.145981,..."
7,1.0,41.256052,-82.616032,"{'type': 'Point', 'coordinates': [-82.616032, ..."
8,2.0,42.409853,-71.256461,"{'type': 'Point', 'coordinates': [-71.256461, ..."
11,1.0,32.781078,-96.797111,"{'type': 'Point', 'coordinates': [-96.797111, ..."


In [59]:
df_clean=pd.concat([df, off], axis=1)[['name', 'lat', 'lng', 'oficina_principal', 'totalOffices']]

In [60]:
df_clean=df_clean.dropna()

In [63]:
df1 =df_clean.reset_index()

In [145]:
import folium 
map = folium.Map(location = [37.0902,95.7129])
locations = df1[['lat','lng']]
locationlist = locations.values.tolist()
for point in range(0,len(locationlist)):
    folium.Marker(locationlist[point], popup=df1['name'][point]).add_to(map)
map

In [72]:
collection1 = db['States']

In [133]:
enf = pd.read_csv('C:/Users/Data Analyst/Desktop/Mongo project/state-comparisons-vital-statistics-and-health.csv')

In [134]:
enf.head()

,Area Name,Year,Variable,Value,Rank,Note,City,Zip,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,Iowa,2007,Median Age In Years,38.0,NaN,NaN,Iowa,70647,LA,30.24321,-93.00386,-6,1,30.243213
1,Maryland,2007,Median Age In Years,37.0,NaN,NaN,Maryland,12116,NY,42.53859,-74.92185,-5,1,42.538589
2,Tennessee,2007,Median Age In Years,38.0,NaN,NaN,Tennessee,62374,IL,40.40355,-90.88340,-6,1,40.403553
3,Nebraska,2008,Median Age In Years,36.0,NaN,NaN,Nebraska,47262,IN,39.00130,-85.61991,-5,0,39.001301
4,Nevada,2008,Median Age In Years,36.0,NaN,NaN,Nevada,50201,IA,42.02602,-93.44840,-6,1,42.026021


In [137]:
clean = enf[((enf.Variable=='Cancer Deaths Per 100') & (enf.Value < 150 ) ) | 
            ((enf.Variable=='Cases Of HIV Reported') & (enf.Value < 100 )) | 
            ((enf.Variable=='Cases Of Syphilis Reported')& (enf.Value < 100 ))|
            ((enf.Variable=='Motor Vehicle Deaths Per 100')& (enf.Value < 20 ))
        ]

In [141]:
clean1 = clean.reset_index()

In [144]:
map = folium.Map(location = [37.0902,95.7129])
locations = clean1[['Latitude','Longitude']]
locationlist = locations.values.tolist()
for point in range(0,len(locationlist)):
    folium.Marker(locationlist[point], popup=clean1['Area Name'][point]).add_to(map)
map

Conclusion : The best Area to have a comany near the biggest ones is Baton rouge near 'Affiliated computed services' or in Boston near ' Novel'